In [9]:
import tensorflow as tf

In [10]:
(train_images, _), (_, _) = tf.keras.datasets.fashion_mnist.load_data()
train_images = (train_images.astype("float32") - 127.5) / 127.5
train_images = tf.expand_dims(train_images, -1)
train_data = tf.data.Dataset.from_tensor_slices(train_images).shuffle(10000).batch(32).prefetch(tf.data.AUTOTUNE)

In [11]:
from tensorflow.keras import layers, models, losses, metrics


# === 2. DISCRIMINATOR ===
def build_discriminator():
    inp = layers.Input(shape=(28, 28, 1))
    x = layers.Conv2D(64, 5, strides=2, padding='same')(inp)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(128, 5, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(1)(x)  # No sigmoid, logits output
    return models.Model(inp, x, name="Discriminator")


# === 3. GENERATOR ===
def build_generator(latent_dim=100):
    inp = layers.Input(shape=(latent_dim,))
    x = layers.Dense(7 * 7 * 128, use_bias=False)(inp)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.Reshape((7, 7, 128))(x)

    x = layers.Conv2DTranspose(64, 4, strides=2, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    '''
    Dropout randomly deactivates neurons during training, 
    helping the generator avoid overfitting to early discriminator 
    feedback and encouraging it to explore more varied outputs — 
    improving sample diversity and training robustness.
    '''
    x = layers.Dropout(0.3)(x)  # Improvement

    x = layers.Conv2DTranspose(1, 4, strides=2, padding='same', activation='tanh')(x)
    return models.Model(inp, x, name="Generator")

# === 4. DCGAN TRAINING ===
class DCGAN(models.Model):
    def __init__(self, discriminator, generator, latent_dim, label_flipping_rate=0.0):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.label_flipping_rate = label_flipping_rate

    def compile(self, d_optimizer, g_optimizer):
        super().compile()
        self.loss_fn = losses.BinaryCrossentropy()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.d_loss_metric = metrics.Mean(name="d_loss")
        self.g_loss_metric = metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]
        random_latent = tf.random.normal((batch_size, self.latent_dim))

        # === DISCRIMINATOR TRAINING ===
        with tf.GradientTape() as d_tape:
            fake_images = self.generator(random_latent, training=True)
            '''
            Adding noise to real images prevents the discriminator
            from memorizing the training data or becoming overconfident.
            This keeps the training signal meaningful for the generator
            and delays mode collapse.
            '''
            real_images += tf.random.normal(tf.shape(real_images), mean=0.0, stddev=0.05)  # Improvement
            real_preds = self.discriminator(real_images, training=True)
            fake_preds = self.discriminator(fake_images, training=True)

            # Smoothed labels + noise
            real_labels = tf.random.uniform(tf.shape(real_preds), minval=0.9, maxval=1.0)
            fake_labels = tf.random.uniform(tf.shape(fake_preds), minval=0.0, maxval=0.1)

            # === LABEL FLIPPING ===
            if self.label_flipping_rate > 0.0:
                real_flip_mask = tf.random.uniform(tf.shape(real_labels)) < self.label_flipping_rate
                fake_flip_mask = tf.random.uniform(tf.shape(fake_labels)) < self.label_flipping_rate
                real_labels = tf.where(real_flip_mask, tf.zeros_like(real_labels), real_labels)
                fake_labels = tf.where(fake_flip_mask, tf.ones_like(fake_labels), fake_labels)

            d_real_loss = self.loss_fn(real_labels, real_preds)
            d_fake_loss = self.loss_fn(fake_labels, fake_preds)
            d_loss = 0.5 * (d_real_loss + d_fake_loss)

        d_grads = d_tape.gradient(d_loss, self.discriminator.trainable_variables)
        self.d_optimizer.apply_gradients(zip(d_grads, self.discriminator.trainable_variables))

        # === GENERATOR TRAINING ===
        with tf.GradientTape() as g_tape:
            fake_images = self.generator(random_latent, training=True)
            fake_preds = self.discriminator(fake_images, training=False)
            g_loss = self.loss_fn(tf.ones_like(fake_preds), fake_preds)

        g_grads = g_tape.gradient(g_loss, self.generator.trainable_variables)
        self.g_optimizer.apply_gradients(zip(g_grads, self.generator.trainable_variables))

        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {"d_loss": self.d_loss_metric.result(), "g_loss": self.g_loss_metric.result()}


In [12]:
import matplotlib.pyplot as plt

In [13]:
class GANMonitor(tf.keras.callbacks.Callback):
    def __init__(self, num_images=5, latent_dim=100, exp_name="default"):
        super().__init__()
        self.num_images = num_images
        self.latent_dim = latent_dim
        self.exp_name = exp_name

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % 5 == 0:
            z = tf.random.normal((self.num_images, self.latent_dim))
            generated = self.model.generator(z, training=False)
            imgs = (generated * 127.5 + 127.5) / 255.0

            # Create subfolder for this experiment
            save_dir = os.path.join("generated", self.exp_name)
            os.makedirs(save_dir, exist_ok=True)
            image_path = os.path.join(save_dir, f"epoch_{epoch + 1}.png")

            plt.figure(figsize=(15, 3))
            for i in range(self.num_images):
                plt.subplot(1, self.num_images, i + 1)
                plt.imshow(imgs[i, :, :, 0], cmap='gray')
                plt.axis('off')

            plt.savefig(image_path)
            plt.close()
            print(f"Wygenerowano próbki zapisane do: {image_path}")


In [14]:
import os

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers


def run_experiment(loss_fn, d_lr, g_lr, g_decay=None, d_decay=None, exp_name="default", epochs=50, label_flipping_rate=0.0):
    tf.keras.backend.clear_session()

    print(f"\nRozpoczynam eksperyment: {exp_name}")
    if d_decay:
        print(f"Używam ExponentialDecay dla generatora: initial_lr={d_decay[0]}, decay_steps={d_decay[1]}, decay_rate={d_decay[2]}")
    
    if g_decay:
        print(f"Używam ExponentialDecay dla dyskryminatora: initial_lr={g_decay[0]}, decay_steps={g_decay[1]}, decay_rate={g_decay[2]}")

    if not (g_decay or g_decay):
        print(f"Stałe learning rate - D: {d_lr}, G: {g_lr}")
    print(f"Funkcja straty: {type(loss_fn).__name__}")
    print(f"Odwracanie etykiet: {label_flipping_rate * 100:.1f}%")

    disc = build_discriminator()
    gen = build_generator()
    dcgan = DCGAN(disc, gen, latent_dim=100, label_flipping_rate=label_flipping_rate)

    if d_decay:
        d_lr = tf.keras.optimizers.schedules.ExponentialDecay(*d_decay)

    if g_decay:
        g_lr = tf.keras.optimizers.schedules.ExponentialDecay(*g_decay)

    dcgan.compile(
        d_optimizer=optimizers.Adam(learning_rate=d_lr, beta_1=0.5),
        g_optimizer=optimizers.Adam(learning_rate=g_lr, beta_1=0.5)
    )
    dcgan.loss_fn = loss_fn

    callbacks = [GANMonitor(num_images=5, latent_dim=100, exp_name=exp_name)]

    history = dcgan.fit(
        train_data,
        epochs=epochs,
        verbose=1,
        callbacks=callbacks
    )

    trained_epochs = len(history.history['g_loss'])
    print(f"✅ Zakończono trening: {trained_epochs} epok (max={epochs})")

    d_losses = history.history['d_loss']
    g_losses = history.history['g_loss']

    os.makedirs("plots", exist_ok=True)
    plot_dir = os.path.join("plots", exp_name)
    os.makedirs(plot_dir, exist_ok=True)
    plot_path = os.path.join(plot_dir, "loss_plot.png")
    plt.figure()
    plt.plot(d_losses, label='Discriminator Loss')
    plt.plot(g_losses, label='Generator Loss')
    plt.title(f"Loss - {exp_name}")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.savefig(plot_path)
    plt.close()
    print(f"Wykres strat zapisano do: {plot_path}")



# Badania mnist z dobranymi parametrami


In [17]:
# === 6. RUN EXPERIMENTS ===

base_epochs = 75

g_lr = 2e-4
d_lr = 1e-4

run_experiment(
    loss_fn=losses.BinaryCrossentropy(),
    d_lr=d_lr,
    g_lr=g_lr,
    exp_name=f"improved_{base_epochs}_d_2e-4_g_1e-4",
    epochs=base_epochs,
)



Rozpoczynam eksperyment: improved_75_d_2e-4_g_1e-4
Stałe learning rate - D: 0.0001, G: 0.0002
Funkcja straty: BinaryCrossentropy
Odwracanie etykiet: 0.0%
Epoch 1/75
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 108s 57ms/step - d_loss: 0.7236 - g_loss: 0.9671
Epoch 2/75
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 108s 57ms/step - d_loss: 0.6892 - g_loss: 0.8454
Epoch 3/75
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 108s 57ms/step - d_loss: 2.5069 - g_loss: 0.8294
Epoch 4/75
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 108s 57ms/step - d_loss: 7.9725 - g_loss: 1.1921e-07
Epoch 5/75
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - d_loss: 7.9722 - g_loss: 1.1921e-07Wygenerowano próbki zapisane do: generated/improved_75_d_2e-4_g_1e-4/epoch_5.png
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 108s 58ms/step - d_loss: 7.9722 - g_loss: 1.1921e-07
Epoch 6/75
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 108s 58ms/step - d_loss: 7.9692 - g_loss: 1.1921e-07
Epoch 7/75
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 108s 58ms/step - d_loss: 7.9744 - g_loss: 1.4658e-05
Epoch 8/75
1875/1875 ━━━━━━━━━